<a href="https://colab.research.google.com/github/ankshah131/localsolve-open/blob/main/NBR_plus_Compute_gee.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook demonstrates how to process Sentinel-2 satellite imagery to calculate the Normalized Burn Ratio (NBR) and the delta Normalized Burn Ratio (dNBR) using the Google Earth Engine (GEE) Python API.

- The **Normalized Burn Ratio (NBR) plus** is a remote sensing index used to identify areas affected by fire. It is enhanced version of NBR which is computed using the near-infrared (NIR), shortwave-infrared (SWIR) bands,Green and Blue bands which are sensitive to vegetation and burned areas.

- The **delta Normalized Burn Ratio (dNBR)** is the difference between pre-fire and post-fire NBR plus values, providing an effective metric for assessing burn severity.


## Workflow
1. Define the region of interest using a GeoJSON polygon.
2. Filter Sentinel-2 imagery by date and cloud cover.
3. Compute the NBR plus and dNBR plus indices.
4. Visualize the results with `geemap`

## Prerequisites

To run this notebook, you will need to have a Google Earth Engine account ([Sign up here](https://earthengine.google.com/)).

You will also need to ensure you have the necessary tools and libraries installed in a clean virtual environment. Follow the steps below to create and set up a virtual environment:  

### Install the dependencies

#### Using `venv` (Python Standard Library)
1. **Create a virtual environment**:  
Ensure the desired Python version (e.g., Python 3.12) is installed on your system. Then run:

```bash  
   python3 -m venv nbr-env
```
This creates a directory named nbr-env that contains the virtual environment.

1. **Activate the virtual environment**

- On macOS/Linux:
```bash

source nbr-env/bin/activate
```
- On Windows (Command Prompt):
```bash
nbr-env\Scripts\activate
```

3. **Install dependencies from requirements.txt**
```bash
pip install -r requirements.txt
```

####  Using conda (Anaconda/Miniconda)
1. Create a new Conda environment:

```bash
conda create --name nbr-env python=3.12
```
2. Activate the environment:

```bash
conda activate nbr-env
```

3. Use the following command to install dependencies:

```bash
pip install -r requirements.txt
```


Now you are all set up, let's start !

## Imports

In [1]:
import ee
import json
import geemap

In [2]:
# Authenticate
ee.Authenticate()
# Initialize the Earth Engine API
ee.Initialize(project='ee-subodhpaudel123')

## Define the Region of Interest

In [4]:
# Load GeoJSON and and convert to Earth Engine geometry
def geojson_to_ee(file_path):
    with open(file_path, "r") as f:
        geojson_data = json.load(f)
    roi = ee.Geometry.Polygon(geojson_data["features"][0]["geometry"]["coordinates"])
    return roi


# Change the Geojson to your own need:
geojson_path = "/content/polygon.json"
roi = geojson_to_ee(geojson_path)
print("ROI set successfully:", roi.getInfo())


ROI set successfully: {'type': 'Polygon', 'coordinates': [[[-118.717712, 34.241424], [-118.712219, 33.973221], [-118.15741, 33.983465], [-118.166473, 34.237918], [-118.717712, 34.241424]]]}


## Data processing

As of now the NBR and dNBR is computed using only Sentinel-2 imagery. Implementation with Landsat 8 will follow.

In [5]:
# Sentinel-2 Surface Reflectance
collection = "COPERNICUS/S2_SR_HARMONIZED"


def mask_s2_clouds(image):
    """Masks clouds in a Sentinel-2 image using the QA band.

    Args:
        image (ee.Image): A Sentinel-2 image.

    Returns:
        ee.Image: A cloud-masked Sentinel-2 image.
    """
    qa = image.select("QA60")

    # Bits 10 and 11 are clouds and cirrus, respectively.
    cloud_bit_mask = 1 << 10
    cirrus_bit_mask = 1 << 11
    #water_layer = ee.ImageCollection("GOOGLE/DYNAMICWORLD/V1").filterBounds(roi).mean()

    # Both flags should be set to zero, indicating clear conditions.
    cloud_mask = qa.bitwiseAnd(cloud_bit_mask).eq(0).And(qa.bitwiseAnd(cirrus_bit_mask).eq(0))
    #water_mask = water_layer.select("water").eq(0)


    return image.updateMask(cloud_mask).divide(10000)


# Define a function to calculate NBR
def calculate_nbr(image):
    nbr_plus = image.expression(
    "(B12A-B8A-B3-B2)/(B12A+B8A+B3+B2)",
          {
            "B12A": image.select("B12"),   # Shortwave Infrared (SWIR)
            "B8A": image.select("B8A"),   # Narrow Near-Infrared (NIR2)
            "B3": image.select("B3"),   # Green band
            "B2": image.select("B2")     # Blue band
        }
    ).rename("NBR_PLUS")
    return image.addBands(nbr_plus)


# Retrieve Sentinel-2 images for a specific time range and region
def get_sentinel2_nbr(roi, start_date, end_date):
    sentinel2 = (
        ee.ImageCollection(collection)
        .filterBounds(roi)
        .filterDate(
            start_date, end_date
        )  # Use the full date range for pre- and post-event
        .map(mask_s2_clouds)
        .map(calculate_nbr)

    )
    # Check if the collection has valid images
    count = sentinel2.size().getInfo()
    if count == 0:
        raise ValueError(
            f"No images found for the date range {start_date} to {end_date}"
        )

    return sentinel2.mean().clip(roi)

In [6]:
# Define the pre-event and post-event dates
pre_fire_start_date = "2024-12-01"
pre_fire_end_date = "2025-01-01"
post_fire_start_date = "2025-01-10"
post_fire_end_date = "2025-01-20"

pre_event_image = get_sentinel2_nbr(roi, pre_fire_start_date, pre_fire_end_date)
print(pre_event_image.getInfo())

post_event_image = get_sentinel2_nbr(roi, post_fire_start_date, post_fire_end_date)
print(post_event_image.getInfo())

{'type': 'Image', 'bands': [{'id': 'B1', 'data_type': {'type': 'PixelType', 'precision': 'float', 'min': 0, 'max': 6.553500175476074}, 'dimensions': [3, 2], 'origin': [-120, 33], 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, {'id': 'B2', 'data_type': {'type': 'PixelType', 'precision': 'float', 'min': 0, 'max': 6.553500175476074}, 'dimensions': [3, 2], 'origin': [-120, 33], 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, {'id': 'B3', 'data_type': {'type': 'PixelType', 'precision': 'float', 'min': 0, 'max': 6.553500175476074}, 'dimensions': [3, 2], 'origin': [-120, 33], 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, {'id': 'B4', 'data_type': {'type': 'PixelType', 'precision': 'float', 'min': 0, 'max': 6.553500175476074}, 'dimensions': [3, 2], 'origin': [-120, 33], 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, {'id': 'B5', 'data_type': {'type': 'PixelType', 'precision': 'float', 'min': 0, 'max': 6.553500175476074}, 'dimensions': [3, 2], 'origin

In [7]:
#calculate dnbr
dnbr = (
    pre_event_image.select("NBR_PLUS")
      .subtract(post_event_image.select("NBR_PLUS"))
      .multiply(1000).rename("dNBR")
)
#dnbr = dnbr.divide(1000)

In [8]:
print(dnbr.getInfo())

{'type': 'Image', 'bands': [{'id': 'dNBR', 'data_type': {'type': 'PixelType', 'precision': 'float'}, 'dimensions': [3, 2], 'origin': [-120, 33], 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}]}


## Mapping the results

In [28]:
Map = geemap.Map()
Map.centerObject(roi, 10)

boundary_style = {
    "color": "ffffff",  # White color
    "strokeWidth": 5,  # Stroke width of 5
}

Map.addLayer(roi, boundary_style, "Study Area")

# Define visualization parameters for NBR (display range, color palette)
nbr_vis = {"min": -1, "max": 1, "palette": ["blue", "white", "green"]}

# Add pre-event and post-event NBR images to the map
Map.addLayer(pre_event_image.select("NBR_PLUS"), nbr_vis, "Pre-Fire NBR")
Map.addLayer(post_event_image.select("NBR_PLUS"), nbr_vis, "Post-Fire NBR")

In [32]:
# Corrected thresholds for dNBR classification (reflecting typical burn severity interpretation)
thresholds =  [-100, 100, 270, 440, 660]  # These are used for burn severity classification

# Apply classification based on these thresholds
classified = (
    dnbr.lt(thresholds[0])  # Enhanced Regrowth (< -0.1)
    .add(dnbr.gte(thresholds[0]).And(dnbr.lt(thresholds[1])).multiply(2))  # Unburned (-0.1 to 0.1)
    .add(dnbr.gte(thresholds[1]).And(dnbr.lt(thresholds[2])).multiply(3))  # Low Severity (0.1 to 0.27)
    .add(dnbr.gte(thresholds[2]).And(dnbr.lt(thresholds[3])).multiply(4))  # Moderate-low Severity (0.27 to 0.44)
    .add(dnbr.gte(thresholds[3]).And(dnbr.lt(thresholds[4])).multiply(5))  # Moderate-high Severity (0.44 to 0.66)
    .add(dnbr.gte(thresholds[4]).multiply(6))  # High Severity (> 0.66)
    .toInt()
)

# Visualization palette based on classification (colors should align with severity)
classified_vis = {
    "min": 0,
    "max": 5,  # Total number of classes
    "palette": ["#7a8737", "#acbe4d", "#0ae042", "#fff70b", "#ffaf38", "#ff641b"],  # Color palette for classification
}

# Add the classified dNBR image to the map
Map.addLayer(classified, classified_vis, "dNBR Classified")

# Add the legend to the map (this is the only legend)
# Check if the legend already exists and clear it
if hasattr(Map, 'legend_added'):
    Map.remove_legend()  # Removes the legend layer before adding a new one

# Add the legend to the map (this is the only legend)
Map.add_legend(
    title="dNBR Classes",
    keys=["Enhanced Regrowth", "Unburned", "Low Severity", "Moderate-low Severity", "Moderate-high Severity", "High Severity"],
    colors=["#7a8737", "#acbe4d", "#0ae042", "#fff70b", "#ffaf38", "#ff641b"],
)

# Set a flag to prevent multiple legends
Map.legend_added = True

In [30]:
print(classified.reduceRegion(ee.Reducer.frequencyHistogram(), roi, 30).getInfo())


{'dNBR': {'1': 191425.24313725487, '2': 1686814.85882353, '3': 111246.30980392155, '4': 606, '5': 103, '6': 11}}


In [31]:
Map

Map(center=[34.10907918494858, -118.44087175579278], controls=(WidgetControl(options=['position', 'transparent…